Experiment with OpenCV for finding the pool table edges within an image

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import numpy.polynomial.polynomial as poly
import math
from Utils import *
import copy
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Load the calibration and test image
dirCalImages = "AmcrestCamera/Checkerboards"

LoadCalibration(dirCalImages)
    
files, dir = FindImageFilesAndDir(subdir=dirCalImages)
filenameTest = files[0]

# Hack-in some ROI tweeking (xywh)
roiNew = (275, 976, 1461, 2280)
SetCalibrationROI(roiNew)

imageTest = cv.imread(filenameTest)
imageTestClipped, imageTestUnclipped = CalibrateImage(imageTest)

ImgShow([imageTestUnclipped, imageTestClipped], 100)

In [ ]:
# Load the test image
dirImages = "AmcrestCamera/Session5"
filename, _ = FindImageFileAndDir(dirImages, "table44.jpg")

print(f'Loading and calibrating file {filename}')
imageOriginal = cv.imread(filename)
imageOriginalClipped, imageOriginalUnclipped = CalibrateImage(imageOriginal)

TX_Load(dirImages)
            
ImgShow([imageOriginalUnclipped, imageOriginalClipped], 100)

imageOriginal = imageOriginalClipped

In [ ]:
# Load the saved transform
TX_Load(dirImages)

# Build a mask for the playable area
imageFelt = TX_MaskPlayableArea(imageOriginal)

ImgShow([imageOriginal, imageFelt], 100)

In [ ]:
# Find the balls, rough first pass

ballDiameter2D = int(2.25 * 100)
ballRadius2D = round(ballDiameter2D / 2)
print(f'ballRadius2D={ballRadius2D} ballDiameter2D={ballDiameter2D}')

def Distance(pts):
    dx = pts[1][0] - pts[0][0]
    dy = pts[1][1] - pts[0][1]
    sumOfSquares = dx*dx + dy*dy
    return math.sqrt(sumOfSquares)


imageFeltG = cv.cvtColor(imageFelt, cv.COLOR_BGR2GRAY)
imagePlaneGEq = cv.equalizeHist(imageFeltG)

# Hough Circles. 
imageInput = imagePlaneGEq
print("imageInput.shape = {0}".format(imageInput.shape))

def FindBalls(imageInput, bAltMode, blur, p1, p2, minDist, minRadius, maxRadius):    
    if bAltMode:
        mode = cv.HOUGH_GRADIENT_ALT
    else:
        mode = cv.HOUGH_GRADIENT

    if blur > 0:
        imageBlur = cv.GaussianBlur(imageInput, (blur,blur), cv.BORDER_DEFAULT)
    else:
        imageBlur = imageInput
        
    hcircles = cv.HoughCircles(imageBlur, mode, minDist=minDist, dp=1,
                               param1=p1, param2=p2,
                               minRadius=minRadius, maxRadius=maxRadius)

    # Create an array of points and radii
    balls2D = []
    ballsN = []
    imageBalls = imageOriginal.copy()
    if hcircles is not None:
        hcircles = np.uint16(np.around(hcircles))
        for hcirc in hcircles[0, :]:
            # We have the point in native space
            x, y, r = hcirc
            ptN = [x,y]
            
            # Check the transformed diameter to see if it is sized like a ball (left to right span)
            ballSidesN = [ [x-r, y], [x+r, y] ]
            ballSides2D = TX_ToFlatPoints(ballSidesN)
            dia2D = Distance(ballSides2D)
            diff = abs(ballDiameter2D - dia2D)
            #print(f'diff={diff}')
            #if diff > 35:
            #    continue    # Skip this ball, it is not the right size in 2D space
            
            cv.circle(imageBalls, ptN, r, Color(), 7)
            
            ballN = hcirc
            ballsN.append(ballN)
            
            pt2D = TX_ToFlatPoint(ptN)
            ball2D = [round(pt2D[0]), round(pt2D[1]), r]
            balls2D.append(ball2D)
            #print(f'  ball={ball}')
            
    return balls2D, ballsN, imageBalls


print(f'image.shape={imageInput.shape}')
balls2D = []
ballsN = []
def OnGui(bAltMode, blur, p1, p2, minRadius, maxRadius):
    # Blur size must be odd
    blur2 = blur
    if 0 == (blur2 % 2):
        blur2 -= 1
    minDist = minRadius * 2
    global balls2D
    global ballsN
    global imageBallsN
    balls2D, ballsN, imageBallsN = FindBalls(imageInput, bAltMode, blur2, p1, p2, minDist, minRadius, maxRadius)
    ImgShow([imageInput, imageBallsN],210)            
    print("balls count 2D = {0}".format(len(balls2D)))
    print("balls count N  = {0}".format(len(ballsN)))
    
w = interactive(OnGui,  bAltMode = widgets.Checkbox(value = False),
                        blur=widgets.IntSlider(min=0, max=71, value=4, step=1),
                        p1=widgets.IntSlider(min=1, max=500, value=35, step=1),
                        p2=widgets.IntSlider(min=1, max=500, value=25, step=1),
                        minRadius=widgets.IntSlider(min=1, max=100, value=18, step=1),
                        maxRadius=widgets.IntSlider(min=1, max=400, value=37, step=1),
                       )
display(w)

In [ ]:
# Color Space investigation for ball edge detection

imageInput = imageOriginal.copy()

def Merge(images):
    for idx, image in enumerate(images):
        if 0 == idx:
            imageMerged = image
        else:
            imageMerged = np.add(imageMerged, image)
            imageMerged = np.clip(imageMerged, 0, 255)
    return imageMerged


def MultiPlaneSobel(image, sobelD, sobelK):
    planes = image.shape[2]
    images = []
    for i in range(0, planes):
        imagePlane = image[:, :, i]
        imageSobel = cv.Sobel(imagePlane, ddepth=cv.CV_8UC1, dx=sobelD, dy=sobelD, ksize=sobelK)
        images.append(imageSobel)
    imageEdges = Merge(images)
    return imageEdges


def MultiPlaneLaplacian(image, ksize):
    planes = image.shape[2]
    images = []
    for i in range(0, planes):
        imagePlane = image[:, :, i]
        imageLaplace = cv.Laplacian(imagePlane, ddepth=cv.CV_8UC1, ksize=ksize)
        images.append(imageLaplace)
    imageEdges = Merge(images)
    return imageEdges



def ProcessColorImage(imageClr, sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius): 
    images = [imageClr]
    
    # Group by Sobel then Laplace
    imagesS = []
    imagesL = []
    for i in range(0, imageClr.shape[2]):
        # Get one plane
        imagePlane = imageClr[:, :, i]        
        
        # Do Sobel and Laplacian for it
        imageSobel = cv.Sobel(imagePlane, ddepth=cv.CV_8UC1, dx=sobelD, dy=sobelD, ksize=sobelK)
        imagesS.append(imageSobel)        
        imageLaplace = cv.Laplacian(imagePlane, ddepth=cv.CV_8UC1, ksize=laplacianKernel)
        imagesL.append(imageLaplace)     
        
    # Do the multi-plane for both Sobel and Laplacian
    imagesS.append(MultiPlaneSobel(imageClr, sobelD, sobelK))
    imagesL.append(MultiPlaneLaplacian(imageClr, ksize=laplacianKernel))
    
    images.extend(imagesS)
    images.extend(imagesL)
        
    if not bShowCircles:
        return images
    
    # Circles
    for idx, img in enumerate(images):
        if 3 == len(img.shape) and 1 != img.shape[2]:
            continue
        
        hcircles = cv.HoughCircles(img, cv.HOUGH_GRADIENT, minDist=maxRadius*2, dp=1,
                        param1=p1, param2=p2,
                        minRadius=minRadius, maxRadius=maxRadius)
        
        img = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
        
        # Draw the circles found
        clrCirc = (0,255,0)
        if hcircles is not None:
            hcircles = np.uint16(np.around(hcircles))
            hcircs = hcircles[0, :]
            for hcirc in hcircs:
                # We have the point in native space
                x, y, r = hcirc
                #print(f'x, y, r = {x},{y},{r}')
                ptN = [x,y]                    
                cv.circle(img, ptN, r, clrCirc, 1)
                
        images[idx] = img
    return images
    
    
images = [] # make a global
def OnGui(sobelDK, laplacianKernel, ballNum, bShowCircles, p1, p2, minRadius, maxRadius): 
    # Parse sobelDK
    tokens = sobelDK.split('/')
    sobelD = int(tokens[0])
    sobelK = int(tokens[1])
    
    if laplacianKernel % 2 == 0:
        laplacianKernel += 1
        
    ballIndex = ballNum - 1
           
    global images
    images = []
    for idx, ballN in enumerate(ballsN):    
        
        # Skip this ball?
        if ballIndex > -1 and idx != ballIndex:
            continue
        
        # extract just the circle/rect from the image, leave a padding border
        x,y,r = ballN
        padding = round(r / 2)
        r += padding
        imageBallOnly = imageInput[y-r-1:y+r+1, x-r-1:x+r+1, :]
        imageBGR = imageBallOnly.copy()    
        
        # We will edge-find in all these images, single-plane only
        images.extend(ProcessColorImage(imageBGR, sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius))
        images.extend(ProcessColorImage(cv.cvtColor(imageBGR, cv.COLOR_BGR2HSV), sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius))
        images.extend(ProcessColorImage(cv.cvtColor(imageBGR, cv.COLOR_BGR2HLS), sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius))
         
    
    ImgShow(images, 210, cols=9)
    # <end OnGui>

sobelDKOptions = [  "1/1", "1/3", "1/5", "1/7",
                    "2/1", "2/3", "2/5", "2/7",
                    "3/5", "3/7",
                    "4/5", "4/7",
                    "5/7",
                    "6/7"]
w = interactive(OnGui,
                sobelDK=widgets.Dropdown(options=sobelDKOptions, value="1/1"),
                laplacianKernel=widgets.IntSlider(min=1, max=31, value=2, step=1),
                ballNum=widgets.IntSlider(min=0, max=16, value=15, step=1),                                          
                bShowCircles = widgets.Checkbox(value = True),                
                p1=widgets.IntSlider(min=1, max=500, value=35, step=1),
                p2=widgets.IntSlider(min=1, max=500, value=25, step=1),
                minRadius=widgets.IntSlider(min=1, max=100, value=10, step=1),
                maxRadius=widgets.IntSlider(min=1, max=400, value=24, step=1)
                )
display(w)

In [ ]:
# Further Ball Finding
#  We use the previous HoughCircles operation as the 'gross' find. Now we 
#  try to narrow it down separately for each ball

imageInput = imageOriginal.copy()

def Flood(image, ptSeed, floodTolerance):    
    image = image.copy()
    cols = image.shape[1]
    rows = image.shape[0]

    imageFloodMaskBrdr = np.zeros((rows+2,cols+2,1), np.uint8)
    diff = (floodTolerance,floodTolerance,floodTolerance)
    flags = 8 | cv.FLOODFILL_FIXED_RANGE | 255 << 8
    res = cv.floodFill(image, imageFloodMaskBrdr, ptSeed, (0,0,255),diff,diff, flags)
    imageFloodMask = TrimBorder(imageFloodMaskBrdr, 1)    
    imageFloodMask= cv.bitwise_not(imageFloodMask)
    imageMasked = cv.bitwise_and(image, image, mask=imageFloodMask)    
    return image, imageFloodMask

def FloodMask(image, ptSeed, floodTolerance):
    _, imageMask = Flood(image, ptSeed, floodTolerance)
    image2 = cv.bitwise_and(image, image, mask=imageMask)    
    return image2        

def OnGui(bShowEdges, floodTolerance, sobelDK, blur, p1, p2, minRadius, maxRadius):   
    # Parse sobelDK
    tokens = sobelDK.split('/')
    sobelD = int(tokens[0])
    sobelK = int(tokens[1])
    
    # Blur size must be odd
    blur2 = blur
    if 0 == (blur2 % 2):
        blur2 -= 1     
        
    images = []
    circCounts = []
    for ballN in ballsN:    
        # extract just the circle/rect from the image, leave a padding border
        x,y,r = ballN
        padding = round(r / 2)
        r += padding
        imageBallOnly = imageInput[y-r-1:y+r+1, x-r-1:x+r+1, :]
        imageBallOnly = imageBallOnly.copy()
        
        # Mask out the corner colors
        rows = imageBallOnly.shape[0]
        cols = imageBallOnly.shape[1]
        if floodTolerance > 0:
            imageBallOnly = FloodMask(imageBallOnly, (0,0), floodTolerance)
            imageBallOnly = FloodMask(imageBallOnly, (rows-1,0), floodTolerance)
            imageBallOnly = FloodMask(imageBallOnly, (rows-1,cols-1), floodTolerance)
            imageBallOnly = FloodMask(imageBallOnly, (0,cols-1), floodTolerance)
        
        if blur2 > 0:
            imageBlur = cv.GaussianBlur(imageBallOnly, (blur2,blur2), cv.BORDER_DEFAULT)
        else:
            imageBlur = imageBallOnly
            
        # Edge detection
        imageEdgesClr = cv.Sobel(imageBlur, ddepth=cv.CV_8UC1, dx=sobelD, dy=sobelD, ksize=sobelK)
        imageEdgesGray = cv.cvtColor(imageEdgesClr, cv.COLOR_BGR2GRAY)
        #print(f'imageEdgesClr={imageEdgesClr.shape} imageEdgesGray={imageEdgesGray.shape}')
        
        #imageBallOnlyGray = cv.cvtColor(imageBlur, cv.COLOR_BGR2GRAY)
        #imageBallOnlyGray = cv.equalizeHist(imageBallOnlyGray)
        
        hcircles = cv.HoughCircles(imageEdgesGray, cv.HOUGH_GRADIENT, minDist=maxRadius, dp=1,
                                param1=p1, param2=p2,
                                minRadius=minRadius, maxRadius=maxRadius)
        
        # Draw the circles found
        clrCirc = (0,255,0)
        circCount = 0
        if hcircles is not None:
            hcircles = np.uint16(np.around(hcircles))
            hcircs = hcircles[0, :]
            for hcirc in hcircs:
                # We have the point in native space
                x, y, r = hcirc
                ptN = [x,y]
                
                cv.circle(imageBallOnly, ptN, r, clrCirc, 1)
                #print(f'pt={x},{y} r={r}')
                circCount += 1
        
        circCounts.append(circCount)
        
        images.append(imageBallOnly)
        if bShowEdges:
            images.append(imageEdgesClr)
            images.append(imageEdgesGray)
            images.append()
            
        # <end for each ball>
            
    print(f'circCounts={circCounts}')
    
    ImgShow(images, 150, cols=6)

sobelDKOptions = [  "1/1", "1/3", "1/5", "1/7",
                    "2/1", "2/3", "2/5", "2/7",
                    "3/5", "3/7",
                    "4/5", "4/7",
                    "5/7",
                    "6/7"]

w = interactive(OnGui,  bShowEdges = widgets.Checkbox(value = False),
                        floodTolerance=widgets.IntSlider(min=0, max=71, value=0, step=1),
                        sobelDK=widgets.Dropdown(options=sobelDKOptions, value="1/1"),
                        blur=widgets.IntSlider(min=0, max=71, value=2, step=1),
                        p1=widgets.IntSlider(min=1, max=500, value=35, step=1),
                        p2=widgets.IntSlider(min=1, max=500, value=25, step=1),
                        minRadius=widgets.IntSlider(min=1, max=100, value=10, step=1),
                        maxRadius=widgets.IntSlider(min=1, max=400, value=24, step=1),
                       )
display(w)
